In [1]:
import requests
import pandas as pd

In [2]:
# Define the API URL
url = "https://www.nseindia.com/api/historical/cm/equity?symbol=20MICRONS&series=[%22EQ%22]&from=01-01-2015&to=01-01-2016"

# Define the headers (NSE website requires specific headers)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nseindia.com/',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive'
}

In [3]:
import io
nse_url = "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"
s=requests.Session()
s.headers.update(headers)

url= "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"

r=s.get(nse_url)
s.close()

df_nse = pd.read_csv(io.BytesIO(r.content))



In [4]:
df_nse.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,BE,03-MAY-1995,10,1,INE253B01015,10
2,360ONE,360 ONE WAM LIMITED,EQ,19-SEP-2019,1,1,INE466L01038,1
3,3IINFOLTD,3i Infotech Limited,EQ,22-OCT-2021,10,1,INE748C01038,10
4,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10


In [5]:
new_df=df_nse[["SYMBOL"]]

In [6]:
import numpy as np
ar=np.array(new_df.iloc[330:661])

In [7]:
ar

array([['CENTUM'],
       ['CENTURYPLY'],
       ['CENTURYTEX'],
       ['CERA'],
       ['CEREBRAINT'],
       ['CESC'],
       ['CGCL'],
       ['CGPOWER'],
       ['CHALET'],
       ['CHAMBLFERT'],
       ['CHEMBOND'],
       ['CHEMCON'],
       ['CHEMFAB'],
       ['CHEMPLASTS'],
       ['CHENNPETRO'],
       ['CHEVIOT'],
       ['CHOICEIN'],
       ['CHOLAFIN'],
       ['CHOLAHLDNG'],
       ['CIEINDIA'],
       ['CIGNITITEC'],
       ['CINELINE'],
       ['CINEVISTA'],
       ['CIPLA'],
       ['CLEAN'],
       ['CLEDUCATE'],
       ['CLSEL'],
       ['CMSINFO'],
       ['COALINDIA'],
       ['COASTCORP'],
       ['COCHINSHIP'],
       ['COFFEEDAY'],
       ['COFORGE'],
       ['COLPAL'],
       ['COMPUSOFT'],
       ['COMSYN'],
       ['CONCOR'],
       ['CONCORDBIO'],
       ['CONFIPET'],
       ['CONSOFINVT'],
       ['CONTROLPR'],
       ['CORALFINAC'],
       ['CORDSCABLE'],
       ['COROMANDEL'],
       ['COSMOFIRST'],
       ['COUNCODOS'],
       ['CRAFTSMAN'],
       ['CR

In [9]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import csv

# Define the base URL and headers
base_url = "https://www.nseindia.com/api/historical/cm/equity"

# Define the headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nseindia.com/',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive'
}
all_stock_data = pd.DataFrame()

symbol = ar
stock_data = pd.DataFrame()

for start_year in range(2015, 2025):
    start_date = f"{start_year}-01-01"
    end_date = f"{start_year + 1}-01-01"
    url = base_url.format(symbol=symbol, start_date=start_date, end_date=end_date)

    session = requests.Session()
    session.get("https://www.nseindia.com", headers=headers)
    response = session.get(url, headers=headers)
    print(response)

    if response.status_code == 200:
        data = response.json()
        if 'data' in data:
            df = pd.DataFrame(data['data'])

            required_columns = ['CH_TIMESTAMP', 'CH_OPENING_PRICE', 'CH_TRADE_HIGH_PRICE',
                                'CH_TRADE_LOW_PRICE', 'CH_CLOSING_PRICE', 'CH_TOT_TRADED_QTY', 'CH_TOTAL_TRADES']
            if all(col in df.columns for col in required_columns):
                df = df[required_columns]
                df['SYMBOL'] = symbol
                columns_order = ['SYMBOL'] + [col for col in df.columns if col != 'SYMBOL']
                df = df[columns_order]
                stock_data = pd.concat([stock_data, df], ignore_index=True)
            else:
                print(f"Missing columns in data for symbol {symbol} for the period {start_date} to {end_date}")
        else:
            print(f"No data found for symbol {symbol} for the period {start_date} to {end_date}")
    else:
        print(f"Failed to fetch data for symbol {symbol} for the period {start_date} to {end_date}")

all_stock_data = pd.concat([all_stock_data, stock_data], ignore_index=True)

# Save the concatenated data to a CSV file
output_filename = 'E:\\cei\\NSE_all_stock_data.csv'
all_stock_data.to_csv(output_filename, index=False)

print(f"All data saved to {output_filename}")


<Response [422]>
Failed to fetch data for symbol [['CENTUM']
 ['CENTURYPLY']
 ['CENTURYTEX']
 ['CERA']
 ['CEREBRAINT']
 ['CESC']
 ['CGCL']
 ['CGPOWER']
 ['CHALET']
 ['CHAMBLFERT']
 ['CHEMBOND']
 ['CHEMCON']
 ['CHEMFAB']
 ['CHEMPLASTS']
 ['CHENNPETRO']
 ['CHEVIOT']
 ['CHOICEIN']
 ['CHOLAFIN']
 ['CHOLAHLDNG']
 ['CIEINDIA']
 ['CIGNITITEC']
 ['CINELINE']
 ['CINEVISTA']
 ['CIPLA']
 ['CLEAN']
 ['CLEDUCATE']
 ['CLSEL']
 ['CMSINFO']
 ['COALINDIA']
 ['COASTCORP']
 ['COCHINSHIP']
 ['COFFEEDAY']
 ['COFORGE']
 ['COLPAL']
 ['COMPUSOFT']
 ['COMSYN']
 ['CONCOR']
 ['CONCORDBIO']
 ['CONFIPET']
 ['CONSOFINVT']
 ['CONTROLPR']
 ['CORALFINAC']
 ['CORDSCABLE']
 ['COROMANDEL']
 ['COSMOFIRST']
 ['COUNCODOS']
 ['CRAFTSMAN']
 ['CREATIVE']
 ['CREATIVEYE']
 ['CREDITACC']
 ['CREST']
 ['CRISIL']
 ['CROMPTON']
 ['CROWN']
 ['CSBBANK']
 ['CSLFINANCE']
 ['CTE']
 ['CUB']
 ['CUBEXTUB']
 ['CUMMINSIND']
 ['CUPID']
 ['CYBERMEDIA']
 ['CYBERTECH']
 ['CYIENT']
 ['CYIENTDLM']
 ['DABUR']
 ['DALBHARAT']
 ['DALMIASUG']
 ['DAMODARI